In [1]:
import requests
import pandas as pd
import yfinance as yf
import numpy as np
import pandas_ta as ta
import random
import statsmodels.api as sm
import scipy.optimize as optimization
import datetime

In [2]:
# Replace with your own Alpha Vantage API key
api_key = 'STQ2DO8BVL7FEP77'

# Alpha Vantage API endpoint for listing stock status
url = f'https://www.alphavantage.co/query?function=LISTING_STATUS&apikey={api_key}'

# Make the request to fetch stock listing data
response = requests.get(url)

# Save the data as CSV
with open('us_stocks.csv', 'w') as f:
    f.write(response.text)

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('us_stocks.csv')

# Filter for only active (listed) stocks
active_stocks = df[df['status'] == 'Active']

# Convert the active stock symbols into a list
#listed_stocks = active_stocks['symbol'].tolist()

def get_all_symbols():
    """Gets all the symbols for the S&P 500"""
    df = pd.read_csv('sp500_companies.csv')
    symbols = [symbol for symbol in df['Symbol']]
    return symbols

listed_stocks = get_all_symbols()

In [3]:
NUM_PORTFOLIOS = 10
NUM_TRADING_DAYS = 252
n_stocks = 10

In [4]:
def statistics(weights, returns):
    portfolio_return = np.sum(returns.mean() * weights) * NUM_TRADING_DAYS
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * NUM_TRADING_DAYS, weights)))

    return np.array([portfolio_return, portfolio_volatility, portfolio_return / portfolio_volatility])

def generate_portfolios_markowitz(returns):
    tickers = [stock for stock in returns]
    portfolio_weights = []
    
    w = np.random.random(len(tickers))
    w /= np.sum(w)
    portfolio_weights.append(w)

    # Convert to DataFrame
    weights_df = pd.DataFrame(portfolio_weights, columns=tickers)
    weights_df = weights_df.mean().reset_index()
    weights_df.columns = ['Ticker', 'Weight']

    return weights_df[['Ticker', 'Weight']]

def min_function_sharpe(weights, returns):
    return -statistics(weights, returns)[2]

def optimize_portfolio(returns):
    # Generate initial weights using the new generate_portfolios_markowitz function
    initial_weights_df = generate_portfolios_markowitz(returns)
    initial_weights = initial_weights_df['Weight'].values

    # Define the constraints and bounds
    constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
    bounds = tuple((0, 1) for _ in range(len(initial_weights)))

    # Perform the optimization
    result = optimization.minimize(
        fun=min_function_sharpe,
        x0=initial_weights,
        args=returns,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )

    # Convert the result to a DataFrame
    optimized_weights = pd.DataFrame({
        'Weight': result.x,
        'Ticker': initial_weights_df['Ticker']
    })

    return optimized_weights[['Ticker', 'Weight']]

In [5]:
ff_url = "https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_CSV.zip"

# Function to randomly select stocks from a given list
def create_random_portfolio(stock_list, n_stocks):
    return random.sample(stock_list, n_stocks)


def download_data(stocks):
    # name of the stock (key) - stock values (2010-date) as the values
    periods = ["max", "1y", "6mo", "3mo", "1mo", "5d", "1d"]
    stock_data = {}
    for stock in stocks:
        ticker = yf.Ticker(stock)
        for period in periods:
            stock_data[stock] = ticker.history(period=period)['Close']
            if not stock_data[stock].empty:
                break
                
    return pd.DataFrame(stock_data)

def calculate_return(data):
    log_return = np.log(data/data.shift(1))
    return log_return[1:]

# Function to calculate technical indicators for each stock
def get_stock_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        # Try to fetch data with different periods
        periods = ["max", "1y", "6mo", "3mo", "1mo", "5d", "1d"]  # Fallback periods
        df = None
        
        for period in periods:
            df = stock.history(period=period)
            if not df.empty:
                break  # Exit loop if valid data is found
        
        if df is None or df.empty:
            print(f"No data for {ticker}, skipping...")
            return None

        # Calculate various technical indicators
        df['RSI'] = ta.rsi(df['Close'], length=14)
        df['Daily_Return'] = df['Close'].pct_change()  # Keep this for custom model strategy
        df['Volatility'] = df['Daily_Return'].rolling(window=20).std() * np.sqrt(252)
        df['MACD_Hist'] = ta.macd(df['Close']).iloc[:, 2]  # MACD histogram
        df['ATR'] = ta.atr(df['High'], df['Low'], df['Close'], length=14)  # Average True Range
        df['STOCH_K'] = ta.stoch(df['High'], df['Low'], df['Close']).iloc[:, 0]  # Stochastic %K
        df['OBV'] = ta.obv(df['Close'], df['Volume'])  # On-Balance Volume

        # Static fundamental data (EPS, Dividends)
        eps = stock.info.get('trailingEps', np.nan)
        dividends = stock.info.get('dividendRate', np.nan)
        
        df['EPS'] = eps
        df['Dividends'] = dividends
        df['Trading_Volume'] = df['Volume']
        df['Liquidity'] = df['Trading_Volume'] * df['Close']
        df['Ticker'] = ticker

        # Drop rows with any missing values
        df.dropna(inplace=True)

        # Return the most recent row with the calculated indicators
        return df[['Close', 'RSI', 'Daily_Return', 'Volatility', 'EPS', 'Dividends', 'Trading_Volume', 'Liquidity', 'MACD_Hist', 'ATR', 'STOCH_K', 'OBV', 'Ticker']].iloc[-1:]

    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

# Function to get the latest stock data and merge with Fama-French factors

def prepare_portfolio_data(stock_list):
    portfolio_data = pd.DataFrame()
    
    for ticker in stock_list:
        stock_data = get_stock_data(ticker)
        if stock_data is not None:
            portfolio_data = pd.concat([portfolio_data, stock_data])
    
    # Fama-French factors (this assumes you already have them in 'ff_data')
    ff_data = pd.read_csv(ff_url, skiprows=3)  # Provide the Fama-French URL as before
    ff_data.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
    ff_data = ff_data[ff_data['Date'].str.isdigit()]
    ff_data['Date'] = pd.to_datetime(ff_data['Date'], format='%Y%m%d')
    ff_data.set_index('Date', inplace=True)
    
    # Take only the latest Fama-French factors (since you're dealing with the latest stock data)
    latest_ff_data = ff_data.iloc[-1:]
    
    # Merge portfolio data with the Fama-French factors
    portfolio_data = portfolio_data.merge(latest_ff_data, how='cross')  # Cartesian product with FF data
     
    return portfolio_data

# Equal weight strategy
def equal_weight_strategy(portfolio_data):
    n_stocks = len(portfolio_data)
    portfolio_data['Weight'] = 1 / n_stocks if n_stocks > 0 else 0  # Avoid division by zero
    return portfolio_data[['Ticker', 'Weight']]



def custom_model_weight_strategy(df):
    # Coefficients from the regression model
    intercept = -2.358e-11
    coef_dict = {
        'RSI': -1.171e-12,
        'Volatility': 1.570e-14,
        'EPS': 3.999e-15,
        'Dividends': 1.675e-12,
        'Trading_Volume': 5.768e-20,
        'Liquidity': 2.184e-20,
        'MACD_Hist': 7.380e-14,
        'ATR': 1.500e-13,
        'STOCH_K': -4.266e-29,
        'OBV': -6.439e-22,
        'Mkt-RF': 1.711e-10,
        'SMB': 9.408e-10,
        'HML': 6.266e-10,
        'RF': -1.000e+00,
        'Stock_Return': 1.000e+00  # This refers to the excess stock return
    }

    # Check if the column 'Stock_Return' is present
    if 'Stock_Return' not in df.columns:
        # If not present, use Daily_Return as a substitute
        df['Stock_Return'] = df['Daily_Return']

    # Calculate the predicted excess return for each stock
    df['Predicted_Return'] = (
        intercept +
        df['RSI'] * coef_dict['RSI'] +
        df['Volatility'] * coef_dict['Volatility'] +
        df['EPS'] * coef_dict['EPS'] +
        df['Dividends'] * coef_dict['Dividends'] +
        df['Trading_Volume'] * coef_dict['Trading_Volume'] +
        df['Liquidity'] * coef_dict['Liquidity'] +
        df['MACD_Hist'] * coef_dict['MACD_Hist'] +
        df['ATR'] * coef_dict['ATR'] +
        df['STOCH_K'] * coef_dict['STOCH_K'] +
        df['OBV'] * coef_dict['OBV'] +
        df['Mkt-RF'] * coef_dict['Mkt-RF'] +
        df['SMB'] * coef_dict['SMB'] +
        df['HML'] * coef_dict['HML'] +
        df['RF'] * coef_dict['RF'] +
        df['Stock_Return'] * coef_dict['Stock_Return']  # Use 'Daily_Return' or stock returns here
    )

    # Normalize the predicted returns to sum to 1 (this becomes the stock weight)
    df['Weight'] = df['Predicted_Return'] / df['Predicted_Return'].sum()

    # Return the DataFrame with stock weights
    return df[['Ticker', 'Weight']]

# Function to return the portfolio weights
def  return_weights(stock_list, n_stocks):
    # Step 1: Create random portfolio
    portfolio = create_random_portfolio(stock_list, n_stocks)
    portfolio_data = prepare_portfolio_data(portfolio)
    
    # Step 2: Get weights from different strategies
    equal_weight_portfolio = equal_weight_strategy(portfolio_data)
    custom_model_portfolio = custom_model_weight_strategy(portfolio_data)

    # Step 3: Generate portfolios using the Markowitz model
    x = download_data(portfolio)
    log_returns = calculate_return(x)

    # Step 4: Optimize the portfolio
    optimum = optimize_portfolio(log_returns)
    #optimal_portfolio_weights = optimum['x']

    return equal_weight_portfolio, custom_model_portfolio, optimum

# Function to backtest portfolio
def backtest_portfolio(weights):
    portfolio_returns = []

    tickers = weights['Ticker'].unique()  # Extract tickers from weights DataFrame

    for ticker in tickers:
        stock = yf.Ticker(ticker)
        df = stock.history(period="max")
        if df.empty:
            print(f"No data for {ticker}, skipping...")
            continue

        df['Daily_Return'] = df['Close'].pct_change()
        df.dropna(inplace=True)

        # Check if the ticker is in the weights DataFrame
        if ticker not in weights['Ticker'].values:
            print(f"{ticker} not found in weights, skipping...")
            continue

        # Calculate the weighted returns
        weight = weights.loc[weights['Ticker'] == ticker, 'Weight'].values[0]
        df['Weighted_Return'] = df['Daily_Return'] * weight
        portfolio_returns.append(df['Weighted_Return'])

    if len(portfolio_returns) == 0:
        print("No data available for backtesting.")
        return None

    # Combine all the returns
    portfolio_returns = pd.concat(portfolio_returns, axis=1).sum(axis=1)

    # Calculate evaluation metrics
    cagr = (portfolio_returns.add(1).prod()**(1/len(portfolio_returns)) - 1) * 252
    sharpe_ratio = portfolio_returns.mean() / portfolio_returns.std() * np.sqrt(252)
    volatility = portfolio_returns.std() * np.sqrt(252)
    max_drawdown = (portfolio_returns.cummax() - portfolio_returns).max()

    # Print the evaluation metrics
    #print(f"CAGR: {cagr:.4f}")
    #print(f"Sharpe Ratio: {sharpe_ratio:.4f}")
    #print(f"Volatility: {volatility:.4f}")
    #print(f"Max Drawdown: {max_drawdown:.4f}")

    return {"CAGR": cagr, "Sharpe Ratio": sharpe_ratio, "Volatility": volatility, "Max Drawdown": max_drawdown}
# Example stock tickers
tickers = listed_stocks

# Step 1: Get the portfolio weights
equal_weight_portfolio, custom_model_portfolio, optimal_weight_portfolio = return_weights(tickers, n_stocks)

print("Equal Weight: ", "\n",equal_weight_portfolio)
print("Custom FAMA French Model: ", "\n", custom_model_portfolio)
print("Markowitz: ", "\n",optimal_weight_portfolio)
# Step 2: Perform backtesting
print("Equal Weight Performance: ", backtest_portfolio(equal_weight_portfolio), "\n")
print("Custom FAMA French Model Performance: ", backtest_portfolio(custom_model_portfolio), "\n")
print("Markowitz Model Performance: ", backtest_portfolio(optimal_weight_portfolio), "\n")

  Ticker        Weight
0    TAP  6.359957e-18
1    DHI  5.577535e-02
2    CSX  1.917657e-01
3     BX  0.000000e+00
4   GILD  1.160786e-01
5      D  5.272206e-01
6    TER  0.000000e+00
7   RVTY  1.255270e-02
8    HAL  8.318342e-17
9   TROW  9.660703e-02
Equal Weight:  
   Ticker  Weight
0    TAP     0.1
1    DHI     0.1
2    CSX     0.1
3     BX     0.1
4   GILD     0.1
5      D     0.1
6    TER     0.1
7   RVTY     0.1
8    HAL     0.1
9   TROW     0.1
Custom FAMA French Model:  
   Ticker    Weight
0    TAP  0.074727
1    DHI  0.165695
2    CSX  0.096970
3     BX  0.164750
4   GILD  0.109418
5      D  0.226513
6    TER -0.310097
7   RVTY -0.138584
8    HAL  0.579169
9   TROW  0.031438
Markowitz:  
   Ticker        Weight
0    TAP  6.359957e-18
1    DHI  5.577535e-02
2    CSX  1.917657e-01
3     BX  0.000000e+00
4   GILD  1.160786e-01
5      D  5.272206e-01
6    TER  0.000000e+00
7   RVTY  1.255270e-02
8    HAL  8.318342e-17
9   TROW  9.660703e-02
Equal Weight Performance:  {'CAGR': 0.

In [11]:
import numpy as np
import pandas as pd
import yfinance as yf

def backtest_multiple_portfolios(stock_list, n_stocks, num_portfolios=100):
    equal_weight_metrics = []
    custom_model_metrics = []
    markowitz_metrics = []

    for _ in range(num_portfolios):
        # Step 1: Create a random portfolio
        portfolio = create_random_portfolio(stock_list, n_stocks)
        portfolio_data = prepare_portfolio_data(portfolio)

        # Step 2: Get weights from different strategies
        equal_weight_portfolio = equal_weight_strategy(portfolio_data)
        custom_model_portfolio = custom_model_weight_strategy(portfolio_data)

        # Step 3: Generate portfolios using the Markowitz model
        x = download_data(portfolio)
        log_returns = calculate_return(x)
        markowitz_portfolio = optimize_portfolio(log_returns)

        # Step 4: Backtest each portfolio
        equal_weight_metrics.append(backtest_portfolio(equal_weight_portfolio))
        custom_model_metrics.append(backtest_portfolio(custom_model_portfolio))
        markowitz_metrics.append(backtest_portfolio(markowitz_portfolio))

    # Filter out None values
    equal_weight_metrics = [m for m in equal_weight_metrics if m is not None]
    custom_model_metrics = [m for m in custom_model_metrics if m is not None]
    markowitz_metrics = [m for m in markowitz_metrics if m is not None]

    # Calculate average metrics for each strategy
    def calculate_avg_metrics(metrics_list):
        if not metrics_list:
            return {"CAGR": None, "Sharpe Ratio": None, "Volatility": None, "Max Drawdown": None}
        return {
            "CAGR": np.mean([m["CAGR"] for m in metrics_list]),
            "Sharpe Ratio": np.mean([m["Sharpe Ratio"] for m in metrics_list]),
            "Volatility": np.mean([m["Volatility"] for m in metrics_list]),
            "Max Drawdown": np.mean([m["Max Drawdown"] for m in metrics_list])
        }

    avg_equal_weight_metrics = calculate_avg_metrics(equal_weight_metrics)
    avg_custom_model_metrics = calculate_avg_metrics(custom_model_metrics)
    avg_markowitz_metrics = calculate_avg_metrics(markowitz_metrics)

    # Print the average metrics for each strategy
    print("Equal Weight Strategy Average Metrics:")
    print(f"Average CAGR: {avg_equal_weight_metrics['CAGR']:.4f}")
    print(f"Average Sharpe Ratio: {avg_equal_weight_metrics['Sharpe Ratio']:.4f}")
    print(f"Average Volatility: {avg_equal_weight_metrics['Volatility']:.4f}")
    print(f"Average Max Drawdown: {avg_equal_weight_metrics['Max Drawdown']:.4f}")

    print("\nCustom Model Strategy Average Metrics:")
    print(f"Average CAGR: {avg_custom_model_metrics['CAGR']:.4f}")
    print(f"Average Sharpe Ratio: {avg_custom_model_metrics['Sharpe Ratio']:.4f}")
    print(f"Average Volatility: {avg_custom_model_metrics['Volatility']:.4f}")
    print(f"Average Max Drawdown: {avg_custom_model_metrics['Max Drawdown']:.4f}")

    print("\nMarkowitz Model Strategy Average Metrics:")
    print(f"Average CAGR: {avg_markowitz_metrics['CAGR']:.4f}")
    print(f"Average Sharpe Ratio: {avg_markowitz_metrics['Sharpe Ratio']:.4f}")
    print(f"Average Volatility: {avg_markowitz_metrics['Volatility']:.4f}")
    print(f"Average Max Drawdown: {avg_markowitz_metrics['Max Drawdown']:.4f}")

    return {
        "Equal Weight": avg_equal_weight_metrics,
        "Custom Model": avg_custom_model_metrics,
        "Markowitz Model": avg_markowitz_metrics
    }

# Example usage
tickers = listed_stocks
n_stocks = 10
num_portfolios = 30
average_performance = backtest_multiple_portfolios(tickers, n_stocks, num_portfolios)

/var/folders/v5/mqtnr2p16yvfvqmsjn2t3l140000gn/T/ipykernel_8985/1040605398.py:211: RuntimeWarning: invalid value encountered in scalar power
  cagr = (portfolio_returns.add(1).prod()**(1/len(portfolio_returns)) - 1) * 252


Equal Weight Strategy Average Metrics:
Average CAGR: 0.1029
Average Sharpe Ratio: 0.7459
Average Volatility: 0.1532
Average Max Drawdown: 0.2459

Custom Model Strategy Average Metrics:
Average CAGR: nan
Average Sharpe Ratio: 0.5060
Average Volatility: 0.3212
Average Max Drawdown: 0.4987

Markowitz Model Strategy Average Metrics:
Average CAGR: 0.1047
Average Sharpe Ratio: 0.8104
Average Volatility: 0.1384
Average Max Drawdown: 0.2271
